# 注意：该版分析使用的数据是第一版的数据，丢失率极高。而新版数据不存在这种情况

In [ ]:
### packages for data manipilation ###
import numpy as np 
import pandas as pd 

### packages for plotting ###
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode

### web info extract###
from bs4 import BeautifulSoup
import requests

### other helpers ###
import os, json, gc, datetime
from datetime import timedelta

# just in case that Chinese character cannot be recognised
from matplotlib.font_manager import FontProperties
font_set = FontProperties(fname="/kaggle/input/chinesecharacter/NotoSansHans-Regular.otf",size=15)

# import datasets
record = pd.read_csv('/kaggle/input/bangumi/record_2020_03_10.tsv',delimiter='\t')
user   = pd.read_csv('/kaggle/input/bangumi/user_2020_03_10.tsv'  ,delimiter='\t')

# 用户数据

因为爬记录的时候会顺带爬主页，就可以顺便看看：

In [ ]:
user.head(3)

我设置爬取id从1到529000的用户数据，成功爬取了508825名用户的主页。成功率还蛮高的。

# 记录数据

分析的重头戏，不知道效果咋样：

In [ ]:
record.loc[record.comment.notnull()].head(5)

不知道数据缺失情况怎么样，这里拿Clannad AS当个例子：

In [ ]:
record.loc[record.iid == 876].state.value_counts()

而真实情况是 1820人想看 / 13401人看过 / 399人在看 / 427人搁置 / 145人抛弃。  
我的天，“看过”和“想看”少了不少，蛋疼。 

---
虽然不清楚scrapy的机理，但爬取数据的成败应该是只与网络有关。所以对于分析来说，随机缺失（missing at random，MAR）的假设是有效的。

出于谨慎，看看不同分数（1-10）的丢失率有没有不同：

In [ ]:
mirror_bgm = 'http://mirror.bgm.rin.cat/'
def rating_status(iid):
    """get rating status from the subject 
    return: a dictionary -- {rate: no. of rating}"""
    temp = requests.get(mirror_bgm + f'subject/{iid}')
    temp.encoding = "UTF-8"
    temp = temp.text
    bans = BeautifulSoup(temp, 'html.parser') 
    
    ouput = [eval(i.get_text()[1:-1]) for i in bans.find_all('span', class_= 'count')]
    ouput.reverse()
    return ouput

def loss_rate(iid):
    """calculate % missing for each rate
    return: dictionary with keys 1-10"""
    
    temp = record.loc[record.iid == eval(iid)].rate.value_counts().to_dict()
    temp_dict = {i+1:0 for i in range(10)}
    for key,value in temp.items():
        temp_dict[key]+=value
        
    for idx, val in enumerate(rating_status(iid)):
        temp_dict[idx+1] = 100 * (1 - temp_dict[idx+1]/val) if val != 0 else 0
    
    
    return temp_dict

def get_anime_list(pages):
    """get samples of anime subjects
    return: a list of string"""
    ouput = []
    for count in range(1,pages+1):
        temp = requests.get(mirror_bgm + f'anime/browser?sort=rank&page={count}')
        temp.encoding = "UTF-8"
        temp = temp.text
        bans = BeautifulSoup(temp, 'html.parser') 
        
        ouput += [i['href'][9:] for i in bans.find_all('a', class_ = "subjectCover cover ll")]
    return ouput

In [ ]:
anime_list = get_anime_list(25)
data_dict = {i+1:[] for i in range(10)}
data_dict['iid'] = []
for iid in anime_list:
    temp = loss_rate(iid)
    for key,value in temp.items():
        data_dict[key].append(value)
    data_dict['iid'].append(iid)   
    
loss_df = pd.DataFrame(data_dict).set_index('iid')
loss_df.head(3)

不同分数的丢失率（%）平均值：

In [ ]:
loss_df.mean()

发现低分数段和10分丢失率较高。

不同分数的丢失率标准差：

In [ ]:
loss_df.std()

推测因为低分数段评分相较高分数段偏少，所以标准差大。

### 一些动画的评分丢失率分布
样本大小为600部

In [ ]:
sns.set(rc={'figure.figsize':(10,6)})
loss_df.apply(np.mean, axis = 1).hist(bins = 25)#.mean()

In [ ]:
print(f"平均丢失率为{round(loss_df.apply(np.mean, axis = 1).mean(),2)}%。")

## 总结

本次数据爬取结果一般，作为整体分析的用途而言只能说是勉强可行，没有到推荐的程度。而针对隔别条目的分析就更是完全不可用了。  
另一方面，低分段和10分丢失率相较之下偏高，是个比较有趣的现象，目前没有想到合理的推测。